In [24]:
import chit

%load_ext autoreload
%autoreload 2

chat = chit.Chat.clone("chitimps.json")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
chat.commit("^N/$jupyter")

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Type your message here...')

In [23]:
chat.rm(-1)